# Get data and packages
Use conda_tensorflow_p36 kernel

In [8]:
%%capture
%%bash
pip install --upgrade turicreate
aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_1B.sf datasets/taxi_1B.sf

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d2xlarge' # change this
results_bucket = f"s3://vaex-sagemaker-demo/benchmarks" # change this

name = 'turicreate'
data_path = 'datasets/taxi_1B.sf'
output_file = f'{name}_{instance_type}_results.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {}
single_repetition = 1
statistic_repetition = 5

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks, name):
    results = pd.DataFrame.from_dict(benchmarks, orient='index')
    results.columns = [name]
    return results

def persist():
    gc.collect()
    get_results(benchmarks, name).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, repetitions=1, **kwargs):
    times = []
    for i in range(repetitions):
        start_time = time.time()
        ret = f(df, **kwargs)
        times.append(time.time()-start_time)
    benchmarks[name] = np.mean(times)
    persist()
    print(f"{name} took: {benchmarks[name]}")
    return benchmarks[name]


!mkdir -p results
!mkdir -p datasets
print(f"test for {single_repetition} repetitions for join and groupby and {statistic_repetition} repetitions for statistics")

test for 1 repetitions for join and groupby and 5 repetitions for statistics


# Benchmark

In [3]:
import turicreate as tc
import numpy as np

# Load data
data = tc.SFrame(data_path)
print(f"size: {data.shape[0]} with {data.shape[1]} columns")

size: 1173057927 with 18 columns


In [4]:
def open_file(df=None):
    return tc.SFrame(data_path)

benchmark(open_file, df=data, name='read_file', repetitions=statistic_repetition)

read_file took: 0.0006588935852050781


0.0006588935852050781

In [5]:
def count(df=None):
    return len(df)

benchmark(count, df=data, name='count', repetitions=statistic_repetition)

count took: 6.866455078125e-06


6.866455078125e-06

In [6]:
def skeatch_mean(df):
    return tc.Sketch(df['fare_amount']).mean()

benchmark(skeatch_mean, data, name='skeatch mean', repetitions=statistic_repetition)

skeatch mean took: 52.724819564819335


52.724819564819335

In [7]:
def skeatch_standatd_deviation(df):
    return  tc.Sketch(df['fare_amount']).std()

benchmark(skeatch_standatd_deviation, data, name='skeatch standatd deviation', repetitions=statistic_repetition)

skeatch standatd deviation took: 50.30595235824585


50.30595235824585

In [8]:
def skeatch_frequent_items(df):
    return tc.Sketch(df['passenger_count']).frequent_items()

benchmark(skeatch_frequent_items, data, name='skeatch frequent items', repetitions=statistic_repetition)

skeatch frequent items took: 35.26147451400757


35.26147451400757

In [9]:
def mean(df):
    return df['fare_amount'].mean()

benchmark(mean, df=data, name='mean', repetitions=statistic_repetition)

mean took: 8.204635620117188


8.204635620117188

In [10]:
def standard_deviation(df):
    return df['fare_amount'].std()

benchmark(standard_deviation, df=data, name='standard deviation', repetitions=statistic_repetition)

standard took: 8.599971103668214


8.599971103668214

Unlike other techonlogies, vaex can return columns, or subset of values to explore lazely, 
but becouse many of the other tecnologies crashed at this point, we return a scalar instead.

In [11]:
def mean_of_sum(df):
    return (df['fare_amount'] + df['trip_distance']).mean()

benchmark(mean_of_sum, df=data, name='sum columns mean', repetitions=single_repetition)
benchmarks['sum columns'] =  np.nan

sum columns mean took: 168.5853488445282


In [12]:
def sum_columns(df):
    return df['fare_amount'] + df['trip_distance']

benchmark(sum_columns, df=data, name='sum columns', repetitions=single_repetition)

sum columns took: 0.0004856586456298828


0.0004856586456298828

In [13]:
def mean_of_product(df):
    return (df['fare_amount'] * df['trip_distance']).mean()

benchmark(mean_of_product, df=data, name='product columns mean', repetitions=single_repetition)

product columns mean took: 228.14237141609192


228.14237141609192

In [14]:
def product(df):
    return df['fare_amount'] * df['trip_distance']

benchmark(product, df=data, name='product columns', repetitions=single_repetition)

product columns took: 0.0005006790161132812


0.0005006790161132812

In [ ]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['pickup_longitude']
    phi_1 = df['pickup_latitude']
    theta_2 = df['dropoff_longitude']
    phi_2 = df['dropoff_latitude']
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    return (2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))).mean()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean', repetitions=repetitions)

In [ ]:
def complicated_arithmetic_operation(df):
    theta_1 = df['pickup_longitude']
    phi_1 = df['pickup_latitude']
    theta_2 = df['dropoff_longitude']
    phi_2 = df['dropoff_latitude']
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    return (2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp)))

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation', repetitions=single_repetition)

In [ ]:
def value_counts(df):
    return df['passenger_count'].value_counts()

benchmark(value_counts, df=data, name='value counts', repetitions=single_repetition)

In [ ]:
def groupby_statistics(df):
    return df.groupby(key_column_names = 'passenger_count', 
           operations = {
               'fare_amount_mean':tc.aggregate.MEAN('fare_amount'),
               'fare_amount_std':tc.aggregate.STD('fare_amount'),
               'tip_amount_mean':tc.aggregate.MEAN('tip_amount'),
               'tip_amount_mean':tc.aggregate.STD('tip_amount')
           })

benchmark(groupby_statistics, df=data, name='groupby statistics', repetitions=single_repetition)

In [ ]:
other = groupby_statistics(data)

In [ ]:
def join_data(df, other):
    return df.join(other, on = 'passenger_count')

benchmark(join_data, data, name='join', repetitions=single_repetition, other=other)

In [ ]:
def join_count(df, other):
    return len(df.join(other, on = 'passenger_count'))

benchmark(join_count, data, name='join count', repetitions=single_repetition, other=other)

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [ ]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

In [ ]:
def filter_data(df):
    expr_filter = (df['pickup_longitude'] > long_min)  & (df['pickup_longitude'] < long_max) & \
                  (df['pickup_latitude'] > lat_min)    & (df['pickup_latitude'] < lat_max) & \
                  (df['dropoff_longitude'] > long_min) & (df['dropoff_longitude'] < long_max) & \
                  (df['dropoff_latitude'] > lat_min)   & (df['dropoff_latitude'] < lat_max)
    return df[expr_filter]

benchmark(filter_data, data, name='filter data', repetitions=repetitions)

In [ ]:
filterd = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

In [ ]:
benchmark(filter_data, filterd, name='filterd count', repetitions=repetitions)
benchmark(mean, filterd, name='filterd mean', repetitions=repetitions)
benchmark(standard_deviation, filterd, name='filtered standard deviation', repetitions=repetitions)
benchmark(mean_of_sum, filterd, name ='filtered sum columns mean', repetitions=repetitions)
benchmarks['filtered sum columns'] =  np.nan
benchmark(mean_of_product, filterd, name ='filterd product columns mean', repetitions=repetitions)
benchmarks['filterd product columns'] = np.nan
benchmark(mean_of_complicated_arithmetic_operation, filterd, name='filterd arithmetic operation mean', repetitions=repetitions)
benchmarks['filterd arithmetic operation'] =  np.nan
benchmark(value_counts, filterd, name ='filtered value counts', repetitions=repetitions)
benchmark(groupby_statistics, filterd, name='filtered groupby statistics', repetitions=repetitions)
benchmark(skeatch_mean, filterd, name='filterd skeatch mean', repetitions=repetitions)
benchmark(skeatch_standatd_deviation, filterd, name='skeatch standatd deviation', repetitions=repetitions)
benchmark(skeatch_frequent_items, filterd, name='skeatch frequent items', repetitions=repetitions)
other = groupby_statistics(filterd)
benchmark(join_data, filterd, name='filtered join', repetitions=repetitions, other=other)
benchmark(join_count, filterd, name='filtered join count', repetitions=repetitions, other=other)
print('Done!')

In [ ]:
print('turicreate')
benchmarks